In [1]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/clustering/src')

!pip install pymysql
import prepare

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as scs

from sklearn.feature_selection import SelectKBest, f_regression

X_train, X_test, X_val, y_train, y_test, y_val = prepare.wrangle_zillow(test_size=.10, k=3, thresh=.4)

These columns were dropped.
                          missing count  missing %
airconditioningtypeid             49095   0.680722
architecturalstyletypeid          71916   0.997144
basementsqft                      72072   0.999307
buildingclasstypeid               72122   1.000000
decktypeid                        71508   0.991487
finishedfloor1squarefeet          66101   0.916516
finishedsquarefeet13              72081   0.999432
finishedsquarefeet15              72111   0.999847
finishedsquarefeet50              66101   0.916516
finishedsquarefeet6               71955   0.997684
fireplacecnt                      63861   0.885458
garagecarcnt                      46826   0.649261
garagetotalsqft                   46826   0.649261
hashottuborspa                    70585   0.978689
poolcnt                           56399   0.781994
poolsizesum                       71255   0.987979
pooltypeid10                      71659   0.993580
pooltypeid2                       71048   0.985109
poo



Your target variable is logerror.

1. Is target significantly different for properties in LAC vs OC vs VC?
2. Divide age of house into bins; same question for age bins.
3. What are the top five features for predicting target?
4. Is there a linear relationship between price per square foot and target? (Pearson r)
5.  Compute the mean(logerror) by zipcode and the overall mean(logerror). Run a t-test between the overall mean and the mean for each zip code. What zip codes have an error significantly higher or lower than the expected error?

Answer those questions through a mix of statistical tests and visualizations.


In [2]:
X_train.head()

,bathroomcnt,bedroomcnt,buildingqualitytypeid,finishedsqft,fips,heatingorsystemtypeid,latitude,longitude,propertylandusetypeid,regionidcity,regionidzip,structuretaxvalue,taxvalue,landtaxvalue,taxamount,censustractandblock,95982.0,95983.0,95984.0,95985.0,95986.0,95987.0,95988.0,95989.0,95991.0,95992.0,95993.0,95994.0,95995.0,95996.0,95997.0,95998.0,95999.0,96000.0,96001.0,96002.0,96003.0,96004.0,96005.0,96006.0,...,97088.0,97089.0,97091.0,97092.0,97094.0,97097.0,97098.0,97099.0,97101.0,97104.0,97106.0,97107.0,97108.0,97109.0,97111.0,97113.0,97116.0,97118.0,97119.0,97298.0,97316.0,97317.0,97318.0,97319.0,97323.0,97324.0,97328.0,97329.0,97330.0,97331.0,97344.0,399675.0,livingarearatio,buildinglandvalueratio,taxrate,age,acres,sqftvalue,age_bins,county
parcelid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14477967,3.5,4.0,5.0,3696.0,6059.0,2.0,33.635972,-117.628064,261.0,12773.0,96996.0,350728.0,769896.0,419168.0,7333.00,6.059032e+13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.616000,0.836724,0.009525,28.0,0.137741,208.305195,0,Orange
10802963,2.0,3.0,7.0,1560.0,6037.0,2.0,34.170600,-118.526000,266.0,12447.0,96349.0,178919.0,223648.0,44729.0,2700.44,6.037140e+13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.038525,4.000067,0.012075,38.0,0.929591,143.364103,0,Los Angeles
12905905,2.0,3.0,4.0,1438.0,6037.0,7.0,34.116412,-117.783258,261.0,39308.0,96494.0,117827.0,349786.0,231959.0,4218.81,6.037402e+13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.202792,0.507965,0.012061,66.0,0.162787,243.244784,1,Los Angeles
11323120,3.0,4.0,7.0,3058.0,6037.0,2.0,34.620883,-118.282130,261.0,40227.0,97329.0,331947.0,474204.0,142257.0,5876.82,6.037901e+13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.029013,2.333432,0.012393,17.0,2.419674,155.069980,0,Los Angeles
12012529,1.0,3.0,8.0,428.0,6037.0,2.0,34.102700,-118.349000,266.0,12447.0,96027.0,54159.0,270813.0,216654.0,3228.71,6.037190e+13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.021949,0.249979,0.011922,56.0,0.447658,632.740654,1,Los Angeles


In [3]:
X_train.age.describe()

count    53071.000000
mean        47.861469
std         22.653113
min          2.000000
25%         31.000000
50%         46.000000
75%         63.000000
max        139.000000
Name: age, dtype: float64

In [4]:
lac = y_train[X_train.county=="Los Angeles"]
oc = y_train[X_train.county=="Orange"]
vc = y_train[X_train.county=="Ventura"]

lac_oc = scs.mannwhitneyu(lac, oc)
lac_vc = scs.mannwhitneyu(lac, vc)
vc_oc = scs.mannwhitneyu(vc, oc)

print("p-values comparing means of logerrors\n")
print(f"LAC, OC: {lac_oc[1]}")
print(f"LAC, VC: {lac_vc[1]}")
print(f"VC, OC: {vc_oc[1]}")


p-values comparing means of logerrors

LAC, OC: 0.00030176234103153864
LAC, VC: 0.0025621713578281185
VC, OC: 0.26378036682924066


In [5]:
new = y_train[X_train.age_bins==0] #newer
old = y_train[X_train.age_bins==1]

new_old = scs.mannwhitneyu(new, old)

print("p-values comparing means of logerrors\n")
print(f"Newer, older: {new_old[1]}")

p-values comparing means of logerrors

Newer, older: 0.00020436392296943533


In [8]:
SelectKBest(f_regression, k=5).fit_transform(X_train.iloc[:,:-1], y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


array([[3.50000000e+00, 4.00000000e+00, 3.69600000e+03, 0.00000000e+00,
        2.08305195e+02],
       [2.00000000e+00, 3.00000000e+00, 1.56000000e+03, 0.00000000e+00,
        1.43364103e+02],
       [2.00000000e+00, 3.00000000e+00, 1.43800000e+03, 0.00000000e+00,
        2.43244784e+02],
       ...,
       [3.00000000e+00, 3.00000000e+00, 2.01800000e+03, 0.00000000e+00,
        2.64125372e+02],
       [2.00000000e+00, 1.00000000e+00, 1.39100000e+03, 0.00000000e+00,
        2.46937455e+02],
       [1.00000000e+00, 3.00000000e+00, 1.17700000e+03, 0.00000000e+00,
        4.68189465e+02]])

In [9]:
scs.pearsonr(X_train.sqftvalue, y_train)

(-0.03786884250807705, 2.6192361818347367e-18)

In [ ]:
mean_error = y_train.mean()
zips = set(X_train.regionidzip)
alpha = .01
for zip_ in zips:
  y_zip = y_train[X_train.regionidzip==zip_].values.reshape(1,-1)
  y_mean = np.zeros([1, len(y_zip[0])]) + mean_error
  res = scs.mannwhitneyu(y_zip, y_mean)
  if res[1] < alpha:
    print(f"{zip_}: {res[1]}")